In [2]:
import pandas as pd

df = pd.read_csv('/content/student_admission_record_dirty.csv')

In [ ]:
df

,Name,Age,Gender,Admission Test Score,High School Percentage,City,Admission Status
0,Shehroz,24.0,Female,50.0,68.90,Quetta,Rejected
1,Waqar,21.0,Female,99.0,60.73,Karachi,NaN
2,Bushra,17.0,Male,89.0,NaN,Islamabad,Accepted
3,Aliya,17.0,Male,55.0,85.29,Karachi,Rejected
4,Bilal,20.0,Male,65.0,61.13,Lahore,NaN
...,...,...,...,...,...,...,...
152,Ali,19.0,Female,85.0,78.09,Quetta,Accepted
153,Bilal,17.0,Female,81.0,84.40,Islamabad,Rejected
154,Fatima,21.0,Female,98.0,50.86,Multan,Accepted
155,Shoaib,-1.0,Male,91.0,80.12,Quetta,Accepted


2. Mengecek Data yang Hilang (Missing Value)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    147 non-null    object 
 1   Age                     147 non-null    float64
 2   Gender                  147 non-null    object 
 3   Admission Test Score    146 non-null    float64
 4   High School Percentage  146 non-null    float64
 5   City                    147 non-null    object 
 6   Admission Status        147 non-null    object 
dtypes: float64(3), object(4)
memory usage: 8.7+ KB


In [4]:
df.isna().sum()

,0
Name,10
Age,10
Gender,10
Admission Test Score,11
High School Percentage,11
City,10
Admission Status,10


In [5]:
df.describe()

,Age,Admission Test Score,High School Percentage
count,147.000000,146.000000,146.000000
mean,19.680272,77.657534,75.684726
std,4.540512,16.855343,17.368014
min,-1.000000,-5.000000,-10.000000
25%,18.000000,68.250000,65.052500
50%,20.000000,79.000000,77.545000
75%,22.000000,89.000000,88.312500
max,24.000000,150.000000,110.500000


In [6]:
df['Admission Status'].describe()

,Admission Status
count,147
unique,2
top,Rejected
freq,76


In [7]:
# Mengatasi missing value
for column in df.columns:
    if df[column].dtype == 'object':
        # Jika kolom bertipe object, isi dengan mode
        df[column].fillna(df[column].mode()[0], inplace=True)
    else:
        # Jika kolom bertipe numerik, isi dengan mean
        df[column].fillna(df[column].mean(), inplace=True)

<ipython-input-7-da1a6285f769>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace=True)
<ipython-input-7-da1a6285f769>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [8]:
df.isna().sum()

,0
Name,0
Age,0
Gender,0
Admission Test Score,0
High School Percentage,0
City,0
Admission Status,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    157 non-null    object 
 1   Age                     157 non-null    float64
 2   Gender                  157 non-null    object 
 3   Admission Test Score    157 non-null    float64
 4   High School Percentage  157 non-null    float64
 5   City                    157 non-null    object 
 6   Admission Status        157 non-null    object 
dtypes: float64(3), object(4)
memory usage: 8.7+ KB


3. Mengatasi Duplikat Data

In [10]:
# Mengecek apakah ada duplicate di seluruh kolom
check_duplicate = df.duplicated().sum()

print(f"Jumlah data yang duplikat = {check_duplicate}")

Jumlah data yang duplikat = 7


In [11]:
# Handling duplicate
df = df.drop_duplicates()

In [12]:
# Mengecek duplicate setelah di-handle
handle_duplicate = df.duplicated().sum()

print(f"Jumlah data yang duplikat = {handle_duplicate}")

Jumlah data yang duplikat = 0


Handling Outlier (IQR Method)

In [22]:
# 1. Cek kolom numerik
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
print("Kolom numerik:", numeric_cols)

Kolom numerik: Index(['Age', 'Admission Test Score', 'High School Percentage'], dtype='object')


In [23]:
# 2. Handling outlier dengan IQR method
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


In [24]:
# Cek jumlah outlier
outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
print(f"Outlier pada kolom {col}: {len(outliers)} data")

Outlier pada kolom High School Percentage: 0 data


In [27]:
# 3. Menghapus outlier (bisa juga pakai capping)
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [28]:
# 4. Cek hasil akhir
print("Data shape setelah handling outlier:", df.shape)

Data shape setelah handling outlier: (148, 7)


In [29]:
from sklearn.preprocessing import LabelEncoder

In [30]:
# 1. Cek kolom kategorikal
cat_cols = df.select_dtypes(include=['object', 'category']).columns
print("Kolom kategorikal:", cat_cols)

Kolom kategorikal: Index(['Name', 'Gender', 'City', 'Admission Status'], dtype='object')


In [31]:
# 2. Encoding (contoh pakai Label Encoding)
le = LabelEncoder()

for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

In [32]:
# 3. Cek hasil
df.head()

,Name,Age,Gender,Admission Test Score,High School Percentage,City,Admission Status
0,52,24.0,0,50.0,68.900000,5,1
1,59,21.0,0,99.0,60.730000,1,1
2,11,17.0,1,89.0,75.684726,0,0
3,4,17.0,1,55.0,85.290000,1,1
4,10,20.0,1,65.0,61.130000,2,1
